# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [151]:
import json
def average_rate(file):
    """(str)->float
    This function will read through a specific json file called 'negotiated_rates' to find the allowed amounts for each listed procedure. 
    It will calculate the average allowed amount and return this value as a float rounded to two decimal places.
    
    >>> average_rate('/data/negotiated_rates.json')
    38.67
    
    >>> type(average_rate('/data/negotiated_rates.json'))
    <class 'float'>
    """
    if file != '/data/negotiated_rates.json': # if the wrong json data file is used...
        raise Exception("Please use this json data file: 'negotiated_rates.json'") # ...inform the user to enter the correct file
    
    with open(file) as f: # opens the json file passed through the parameter # set up to close file automatically
        negotiated_rates = json.load(f) # reads json file into a dictionary called 'negotiated_rates'
    
    total_allowed_amount = 0 # set total of allowed amounts to zero
    counter_allowed_amount = 0 # set counter of allowed amounts to zero
    
    out_of_network = negotiated_rates.get('out_of_network') # assigns values in out_of_network key to a list called 'out_of_network'
    
    for oon in out_of_network: # iterate through each item in 'out_of_network'
        allowed_amounts = oon.get('allowed_amounts') # assigns value in allowed_amounts key to 'allowed_amounts'
        
        for aa in allowed_amounts: # iterate through each item in 'allowed_amounts'
            payments = aa.get('payments') # assigns value in payments key to 'payments'
            total_allowed_amount += payments.get('allowed_amount') # adds value from allowed_amount key to total
            counter_allowed_amount += 1 # increases counter by 1
    
    average_allowed_amount = round((total_allowed_amount / counter_allowed_amount), 2) # divide total allowed amounts by the number of allowed amounts...
                                                                                       # ...and round to 2 decimal places
    return average_allowed_amount # return the average of allowed amounts


In [152]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

In [153]:
import doctest
doctest.run_docstring_examples(average_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    average_rate('/data/negotiated_rates.json')
Expecting:
    38.67
ok
Trying:
    type(average_rate('/data/negotiated_rates.json'))
Expecting:
    <class 'float'>
ok


---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [154]:
import json

def get_rate(file, billing_code, service_code):
    """(str, str, str)->float or NoneType
    This function will read through a specific json file called 'negotiated_rates' to find the allowed amount rate for a specified billing code
    and service code. It will return the rate as a float if the codes match or None if the codes do not.
    
    >>> get_rate('/data/negotiated_rates.json', '97110', '11')
    21.51
    
    >>> get_rate('/data/negotiated_rates.json', 'ABCDE', '00')
    """
    if file != '/data/negotiated_rates.json': # if the wrong json data file is used...
        raise Exception("Please use this json data file: 'negotiated_rates.json'") # ...inform the user to enter the correct file
    
    with open(file) as f: # opens the json file passed through the parameter # set up to close file automatically
        negotiated_rates = json.load(f) # reads json file into a dictionary called 'negotiated_rates'
    
    rate = None # set rate to None
    
    out_of_network = negotiated_rates.get('out_of_network') # assigns values in out_of_network key to a list called 'out_of_network'
    
    for oon in out_of_network: # iterate through each item in 'out_of_network'
        oon_billing_code = oon.get('billing_code') # assigns value in billing_code key to 'oon_billing_code'
        allowed_amounts = oon.get('allowed_amounts') # assigns value in allowed_amounts key to 'allowed_amounts'
        
        for aa in allowed_amounts: # iterate through each item in 'allowed_amounts'
            aa_service_code = aa.get('service_code') # assigns value in service_code key to 'aa_service_code'
            payments = aa.get('payments') # assigns value in payments key to 'payments'
            
            if oon_billing_code == billing_code and aa_service_code == service_code: # if the billing and service codes match...
                rate = payments.get('allowed_amount') # ...then assign the value in allowed_amount key to 'rate'
                
    return rate # return the rate
    

In [155]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

In [156]:
import doctest
doctest.run_docstring_examples(get_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_rate('/data/negotiated_rates.json', '97110', '11')
Expecting:
    21.51
ok
Trying:
    get_rate('/data/negotiated_rates.json', 'ABCDE', '00')
Expecting nothing
ok


---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [157]:
from datetime import datetime

def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    """(str, str, str, int, str)->float or NoneType
    This function calls upon the get_rate() function to determine an adjusted rate based on these special rules: when visit dates are on Mondays, 
    rate is reduced to 75%; when visit dates are on Mondays and the patient is 65+ OR just the patient is 65+, rate is reduced to 50%. All other
    conditions means the rate stays the same as before. This adjusted rate (or same rate) is then rounded to two decimal places and returned as 
    a float value (or as a None value if the rate is not found).
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-10-17')
    6.58
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',64,'2022-10-18')
    8.78
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','ABCDE','00',22,'2022-10-17')
    """
    
    rate = get_rate(file, billing_code, service_code) # assign get_rate() value to 'rate' to get the standard rate
    
    visit_date_number = datetime.strptime(visit_date, '%Y-%m-%d').weekday() # takes the visit date and produces the weekday it falls on as a number
    
    if rate == None: # if the standard rate is not available...
        adjusted_rate = None # ...then there is no adjusted rate
        
    elif pat_age >= 65: # else if the patient is 65+...
        adjusted_rate = rate * 0.50 # ...then the standard rate is reduced to 50%
        
    elif visit_date_number == 0: # else if the visit date falls on a Monday (.weekday() produces a 0 for Mondays)...
        adjusted_rate = rate * 0.75 # ...then the standard rate is reduced to 75%
        
    else: # if none of the special rules are met...
        adjusted_rate = rate # ...then the rate remains is not adjusted and stays the same   
    
    if isinstance(adjusted_rate, float): # if the adjusted rate is a float value and not a NoneType...
        adjusted_rate = round(adjusted_rate, 2) # ...then round the adjusted rate to two decimal places
        
    return adjusted_rate # return the adjusted rate

In [158]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

In [159]:
import doctest
doctest.run_docstring_examples(get_adjusted_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-10-17')
Expecting:
    6.58
ok
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',64,'2022-10-18')
Expecting:
    8.78
ok
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','ABCDE','00',22,'2022-10-17')
Expecting nothing
ok


---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [160]:
import json
import csv
from datetime import datetime

def summarize_reimbursement(csv_file, json_file):
    """(str, str)-> dict, dict
    This function reads information from a csv file called 'visits' and the previous json file and consolidates the information by using the 
    get_adjusted_rate() function. If the information in 'visits' matches the information in 'negotiated_rates', then the adjusted rate calculated
    for each row in 'visits' will be added to two separate dictionaries, one based on the months when the visits occurred and one based on the 
    hospitals where the visits occured. 
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-08']
    152.28
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['SLU Hospital']
    409.67
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]
    {'Missouri Baptist': 514.18, 'SSM DePaul': 460.02, 'SLU Hospital': 409.67, 'Barnes Jewish': 485.7, 'Mercy Springfield': 518.59, 'Mercy St. Louis': 380.07}
    """
    if csv_file != '/data/visits.csv': # if the wrong csv file is used...
        raise Exception("Please use this csv data file: '/data/visits.csv'") # ...inform the user to enter the correct file
    
    month_dict = {} # create empty month dictionary
    hosp_dict = {} # create empty hospital dictionary

    with open(csv_file) as f: # open/close 'visits.csv' into 'f'
        csv_reader = csv.reader(f) # read through 'f' and creates csv_reader object
        next(csv_reader) # skips over header row

        for row in csv_reader: # iterates over every row in csv.reader
            rate = get_adjusted_rate(json_file, row[4], row[5], int(row[3]), row[6]) # call upon get_adjusted_rate() and set as 'rate'
            visit_date = datetime.strptime(row[6], '%Y-%m-%d') # converts date found under row[6] from str to datetime.datetime
            
            if rate == None: # if the information from the current row does not produce an actual rate...
                pass # ...skip to next row
            
            else:
                if visit_date.strftime("%Y-%m") not in month_dict: # if the date formatted as YYYY-mm is not in the month dictionary as a key...
                    month_dict[visit_date.strftime("%Y-%m")] = rate # ...then add YYYY-mm as a key and the current rate as its value

                else: # YYYY-mm is in month dictionary already
                    month_dict[visit_date.strftime("%Y-%m")] += rate # add the rate to the existing rate value in the month dictionary

                if row[1] not in hosp_dict: # if the hospital name is not in the hospital dictionary as a key...
                    hosp_dict[row[1]] = rate # ...then add the hospital name as a key and the current rate as its value

                else: # hospital name is in hospital dictionary already
                    hosp_dict[row[1]] += rate # add the rate to the existing rate value in the hospital dictionary

        for month, total_rate in month_dict.items(): # iterate through each key in month dictionary
            month_dict[month] = round(total_rate, 2) # round the value of total rate to two decimal places to account for floating-points

        for hospital, total_rate in hosp_dict.items(): # iterate through each key in hospital dictionary
            hosp_dict[hospital] = round(total_rate, 2) # round the value of total rate to two decimal places to account for floating-points


        return month_dict, hosp_dict # return each dictionary

In [161]:
by_month, by_hospital = summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')
print(by_month)
print(by_hospital)

{'2021-05': 192.38, '2021-03': 378.72, '2021-07': 277.67, '2021-06': 236.53, '2021-11': 229.7, '2021-10': 234.52, '2021-12': 297.87, '2021-04': 337.7, '2021-09': 160.4, '2021-01': 111.91, '2021-02': 158.55, '2021-08': 152.28}
{'Missouri Baptist': 514.18, 'SSM DePaul': 460.02, 'SLU Hospital': 409.67, 'Barnes Jewish': 485.7, 'Mercy Springfield': 518.59, 'Mercy St. Louis': 380.07}


In [162]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)

In [163]:
import doctest
doctest.run_docstring_examples(summarize_reimbursement, globals(), verbose=True)

Finding tests in NoName
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-08']
Expecting:
    152.28
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['SLU Hospital']
Expecting:
    409.67
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]
Expecting:
    {'Missouri Baptist': 514.18, 'SSM DePaul': 460.02, 'SLU Hospital': 409.67, 'Barnes Jewish': 485.7, 'Mercy Springfield': 518.59, 'Mercy St. Louis': 380.07}
ok
